# Cross validation: An Ensemble of Regression with ARIMA errors and Holt-Winters
---

An unweighted ensemble of 

* Regression with new year holiday and (auto) ARIMA errors; and 
* Holt-Winters Exponential Smoothing (statespace formulation)

The ARIMA process modelled is a (1,1,3)(1,0,1,7).  This was determined by auto_arima from the `pmdarima` package.

Note that HW provides no way to easily incorporate special events.

This notebook conducts cross validation of the method using a rolling forecast origin method.

> The Reg with ARIMA error model is based on SWAST's data.  It may be prudent for each trust to fit their own automatic ARIMA model.

**The notebook outputs:**
* MASE, RMSE and MAPE at 7 day intervals from 7 to 84 days.
* 80 and 95% prediction intervals between 7 and 84 days.

These are saved into the folder `results/model_selection/stage1/`

---

# Imports

In [1]:
import pandas as pd
import numpy as np

#error measures
from forecast_tools.metrics import (mean_absolute_scaled_error, 
                                    root_mean_squared_error,
                                    symmetric_mean_absolute_percentage_error)

#models
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.exponential_smoothing import ExponentialSmoothing

import warnings
warnings.filterwarnings('ignore')

In [2]:
import statsmodels as sm
import pmdarima 

print(sm.__version__)
print(pmdarima.__version__)

0.11.0
1.5.2


In [3]:
#to select exceptionally busy days as covariates.
from amb_forecast.feature_engineering import (regular_busy_calender_days)

In [4]:
#custom ensemble class
from amb_forecast.ensemble import (Ensemble, UnweightedVote)

# Data Input

The constants `TOP_LEVEL`, `STAGE`, `REGION`,`TRUST` and `METHOD` are used to control data selection and the directory for outputting results.  

> Output file is `f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv'.csv`.  where metric will be smape, rmse, mase, coverage_80 and coverage_95. Note: `REGION`: is also used to select the correct data from the input dataframe.

In [5]:
TOP_LEVEL = '../../../results/model_selection'
STAGE = 'stage1'
REGION = 'Trust'
METHOD = 'arima-hw'

FILE_NAME = 'Daily_Responses_5_Years_2019_full.csv'

#split training and test data.
TEST_SPLIT_DATE = '2019-01-01'

#second subdivide: train and val
VAL_SPLIT_DATE = '2017-07-01'

#discard data after 2020 due to coronavirus
#this is the subject of a seperate study.
DISCARD_DATE = '2020-01-01'

In [6]:
#read in path
path = f'../../../data/{FILE_NAME}'

In [7]:
def pre_process_daily_data(path, index_col, by_col, 
                           values, dayfirst=False):
    '''
    Daily data is stored in long format.  Read in 
    and pivot to wide format so that there is a single 
    colmumn for each regions time series.
    '''
    df = pd.read_csv(path, index_col=index_col, parse_dates=True, 
                     dayfirst=dayfirst)
    df.columns = map(str.lower, df.columns)
    df.index.rename(str(df.index.name).lower(), inplace=True)
    
    clean_table = pd.pivot_table(df, values=values.lower(), 
                                 index=[index_col.lower()],
                                 columns=[by_col.lower()], aggfunc=np.sum)
    
    clean_table.index.freq = 'D'
    
    return clean_table

In [8]:
clean = pre_process_daily_data(path, 'Actual_dt', 'ORA', 'Actual_Value', 
                               dayfirst=False)
clean.head()

ora,BNSSG,Cornwall,Devon,Dorset,Gloucestershire,OOA,Somerset,Trust,Wiltshire
actual_dt,,,,,,,,,
2013-12-30,415.0,220.0,502.0,336.0,129.0,NaN,183.0,2042.0,255.0
2013-12-31,420.0,236.0,468.0,302.0,128.0,NaN,180.0,1996.0,260.0
2014-01-01,549.0,341.0,566.0,392.0,157.0,NaN,213.0,2570.0,351.0
2014-01-02,450.0,218.0,499.0,301.0,115.0,NaN,167.0,2013.0,258.0
2014-01-03,419.0,229.0,503.0,304.0,135.0,NaN,195.0,2056.0,269.0


## Train Test Split

In [9]:
def ts_train_test_split(data, split_date):
    '''
    Split time series into training and test data
    
    Parameters:
    -------
    data - pd.DataFrame - time series data.  Index expected as datatimeindex
    split_date - the date on which to split the time series
    
    Returns:
    --------
    tuple (len=2) 
    0. pandas.DataFrame - training dataset
    1. pandas.DataFrame - test dataset
    '''
    train = data.loc[data.index < split_date]
    test = data.loc[data.index >= split_date]
    return train, test

In [10]:
train, test = ts_train_test_split(clean, split_date=TEST_SPLIT_DATE)

#exclude data after 2020 due to coronavirus.
test, discard = ts_train_test_split(test, split_date=DISCARD_DATE)

#train split into train and validation
train, val = ts_train_test_split(train, split_date=VAL_SPLIT_DATE)


In [11]:
#amount of training data
train.shape

(1279, 9)

In [12]:
#amount of validation data
val.shape

(549, 9)

# New years day

In [13]:
exceptional = regular_busy_calender_days(train[REGION], quantile=0.99)

In [14]:
new_year = pd.DataFrame({
                         'holiday': 'new_year',
                         'ds': pd.date_range(start=exceptional[0], 
                                             periods=20, 
                                             freq='YS')
                        })

In [15]:
new_year.head()

,holiday,ds
0,new_year,2013-01-01
1,new_year,2014-01-01
2,new_year,2015-01-01
3,new_year,2016-01-01
4,new_year,2017-01-01


# Wrapper classes for statsmodels ARIMA and statsmodels Exponential Smoothing.

Adapter/wrapper classes to enable usage within `Ensemble` class and work with cross validation.

In [16]:
class ARIMAWrapper(object):
    '''
    Facade for statsmodels.statespace
    '''
    def __init__(self, order, seasonal_order, training_index, holidays=None):
        self._order = order
        self._seasonal_order = seasonal_order
        self._training_index = training_index
        self._holidays = holidays

    def _get_resids(self):
        return self._fitted.resid

    def _get_preds(self):
        return self._fitted.fittedvalues
    
    def _encode_holidays(self, holidays, idx):
        dummy = idx.isin(holidays).astype(int)
        dummy = pd.DataFrame(dummy)
        dummy.columns = ['holiday']
        dummy.index = idx
        return dummy

    def fit(self, y_train):
        
        #extend training index
        if len(y_train) > len(self._training_index):

            self._training_index = pd.date_range(start=self._training_index[0], 
                                                 periods=len(y_train),
                                                 freq=self._training_index.freq)
            
        holiday_train = None
        if not self._holidays is None:
            holiday_train = self._encode_holidays(self._holidays, 
                                                  self._training_index)
    
        
        self._model = ARIMA(endog=y_train,
                            exog=holiday_train,
                            order=self._order, 
                            seasonal_order=self._seasonal_order)#,
                            #enforce_stationarity=False)
        
        self._fitted = self._model.fit()
        self._t = len(train)
        
    
    def predict(self, horizon, return_conf_int=False, alpha=0.2):
        '''
        forecast h steps ahead.
        
        Params:
        ------
        h: int
            h-step forecast
        
        return_conf_int: bool, optional (default=False)
            return 1 - alpha PI
        
        alpha: float, optional (default=0.2)
            return 1 - alpha PI
                       
        Returns:
        -------
        np.array
            If return_conf_int = False returns preds only
            
        np.array, np.array
            If return_conf_int = True returns tuple of preds, pred_ints
        '''
        
        #+1 to date range then trim off the first value

        f_idx = pd.date_range(start=self._training_index[-1], 
                              periods=horizon+1,
                              freq=self._training_index.freq)[1:]
        
        #encode holidays if included.
        exog_holiday = None
        if not self._holidays is None:
            exog_holiday = self._encode_holidays(self._holidays, f_idx)
        
    
        forecast = self._fitted.get_forecast(horizon, exog=exog_holiday)
        mean_forecast = forecast.summary_frame()['mean'].to_numpy()
        
        if return_conf_int:
            df = forecast.summary_frame(alpha=alpha)
            pi = df[['mean_ci_lower', 'mean_ci_upper']].to_numpy()
            return mean_forecast, pi
            
        
        else:
            return mean_forecast

    fittedvalues = property(_get_preds)
    resid = property(_get_resids)  

In [17]:
class ExponentialSmoothingWrapper:
    '''
    Facade for statsmodels exponential smoothing models.  This wrapper
    provides a common interface for all models and allow interop with
    the custom time series cross validation code.
    '''
    def __init__(self, trend=False, damped_trend=False, seasonal=None):
        self._trend = trend
        self._seasonal= seasonal
        self._damped_trend = damped_trend

    def _get_resids(self):
        return self._fitted.resid

    def _get_preds(self):
        return self._fitted.fittedvalues

    def fit(self, train):
        '''
        Fit the model
        
        Parameters:
        train: array-like
            time series to fit.
        '''
        self._model = ExponentialSmoothing(endog=train,
                                          trend=self._trend, 
                                          damped_trend=self._damped_trend,
                                          seasonal=self._seasonal)
        self._fitted = self._model.fit()
        self._t = len(train)
    
    def predict(self, horizon, return_conf_int=False, alpha=0.2):
        '''
        Forecast the time series from the final point in the fitted series.
        
        Parameters:
        ----------
        
        horizon: int
            steps ahead to forecast 
            
        return_conf_int: bool, optional (default=False)
            Return prediction interval?  
            
        alpha: float
            Used if return_conf_int=True. 100(1-alpha) interval.
        '''
        
        forecast = self._fitted.get_forecast(horizon)
        
        mean_forecast = forecast.summary_frame()['mean'].to_numpy()
        
        if return_conf_int:
            df = forecast.summary_frame(alpha=alpha)
            pi = df[['mean_ci_lower', 'mean_ci_upper']].to_numpy()
            return mean_forecast, pi        
        else:
            return mean_forecast

    fittedvalues = property(_get_preds)
    resid = property(_get_resids)

# Example of fitting the ensemble
1. Regression with New Year Holiday and Auto ARIMA errors
2. Holt-Winters Exponential Smoothing

The code below demonstrates how to fit the model.

In [18]:
model_1 = ARIMAWrapper(order=(1,1,3), seasonal_order=(1,0,1,7), 
                       training_index=train.index,
                       holidays=new_year['ds'].tolist())

model_2 = ExponentialSmoothingWrapper(trend=True, damped_trend=True, 
                                      seasonal=7)

In [19]:
estimators = {'arima': model_1, 'hw': model_2}
ens = Ensemble(estimators, UnweightedVote())

In [20]:
#fit to training data in chosen region
ens.fit(train[REGION])

In [21]:
#predict 7 days ahead
H = 7
ens_preds = ens.predict(horizon=H)

In [22]:
#view predictions
ens_preds

array([2261.24638683, 2250.59085702, 2148.79400083, 2083.72132816,
       2074.14597754, 2081.28015005, 2122.24263417])

In [23]:
#with prediction intervals
ens_preds, pi = ens.predict(horizon=H, return_conf_int=True)

In [24]:
ens_preds

array([2261.24638683, 2250.59085702, 2148.79400083, 2083.72132816,
       2074.14597754, 2081.28015005, 2122.24263417])

In [25]:
pi

array([[2165.3015595 , 2357.19121417],
       [2147.72563186, 2353.45608217],
       [2041.94187011, 2255.64613154],
       [1973.26005144, 2194.18260487],
       [1960.38309659, 2187.9088585 ],
       [1964.46960816, 2198.09069193],
       [2002.59663442, 2241.88863393]])

# Cross validation functions

`time_series_cv` implements rolling forecast origin cross validation for time series.  
It does not calculate forecast error, but instead returns the predictions, pred intervals and actuals in an array that can be passed to any forecast error function. (this is for efficiency and allows additional metrics to be calculated if needed).

In [26]:
def time_series_cv(model, train, val, horizons, alpha=0.2, step=1):
    '''
    Time series cross validation across multiple horizons for a single model.

    Incrementally adds additional training data to the model and tests
    across a provided list of forecast horizons. Note that function tests a
    model only against complete validation sets.  E.g. if horizon = 15 and 
    len(val) = 12 then no testing is done.  In the case of multiple horizons
    e.g. [7, 14, 28] then the function will use the maximum forecast horizon
    to calculate the number of iterations i.e if len(val) = 365 and step = 1
    then no. iterations = len(val) - max(horizon) = 365 - 28 = 337.
    
    Parameters:
    --------
    model - forecasting model

    train - np.array - vector of training data

    val - np.array - vector of validation data

    horizon - list of ints, forecast horizon e.g. [7, 14, 28] days
    
    alpha - float, optional (default=0.2)
        1 - alpha prediction interval specification

    step -- int, optional (default=1)
            step taken in cross validation 
            e.g. 1 in next cross validation training data includes next point 
            from the validation set.
            e.g. 7 in the next cross validation training data includes next 7 points
            (default=1)
            
    Returns:
    -------
    np.array, np.array, np.array
        - cv_preds, cv_test, cv_intervals
    '''
    
    #point forecasts
    cv_preds = [] 
    #ground truth observations
    cv_actuals = [] 
    #prediction intervals
    cv_pis = []
    
    split = 0

    print('split => ', end="")
    for i in range(0, len(val) - max(horizons) + 1, step):
        split += 1
        print(f'{split}, ', end="")
                
        train_cv = np.concatenate([train, val[:i]], axis=0)
        model.fit(train_cv)
        
        #predict the maximum horizon 
        preds, pis = model.predict(horizon=len(val[i:i+max(horizons)]), 
                                   return_conf_int=True,
                                   alpha=alpha)        
        cv_h_preds = []
        cv_test = []
        cv_h_pis = []
        
        #sub horizon calculations
        for h in horizons:
            #store the h-step prediction
            cv_h_preds.append(preds[:h])
            #store the h-step actual value
            cv_test.append(val.iloc[i:i+h])    
            cv_h_pis.append(pis[:h])
                     
        cv_preds.append(cv_h_preds)
        cv_actuals.append(cv_test)
        cv_pis.append(cv_h_pis)
        
    print('done.\n')        
    return cv_preds, cv_actuals, cv_pis

## Custom functions for calculating CV scores for point predictions and coverage.

These functions have been written to work with the output of `time_series_cv`

In [27]:
def split_cv_error(cv_preds, cv_test, error_func):
    '''
    Forecast error in the current split
    
    Params:
    -----
    cv_preds, np.array
        Split predictions
        
    
    cv_test: np.array
        acutal ground truth observations
        
    error_func: object
        function with signature (y_true, y_preds)
        
    Returns:
    -------
        np.ndarray
            cross validation errors for split
    '''
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = error_func(cv_test[split], cv_preds[split])
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv(cv_preds, cv_test, error_func):
    '''
    Forecast errors by forecast horizon
    
    Params:
    ------
    cv_preds: np.ndarray
        Array of arrays.  Each array is of size h representing
        the forecast horizon specified.
        
    cv_test: np.ndarray
        Array of arrays.  Each array is of size h representing
        the forecast horizon specified.
        
    error_func: object
        function with signature (y_true, y_preds)
        
    Returns:
    -------
    np.ndarray
        
    '''
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error(cv_preds[h], cv_test[h], error_func)
        horizon_errors.append(split_errors)

    return np.array(horizon_errors)

def split_coverage(cv_test, cv_intervals):
    n_splits = len(cv_test)
    cv_errors = []
        
    for split in range(n_splits):
        val = np.asarray(cv_test[split])
        lower = cv_intervals[split].T[0]
        upper = cv_intervals[split].T[1]
        
        coverage = len(np.where((val > lower) & (val < upper))[0])
        coverage = coverage / len(val)
        
        cv_errors.append(coverage)
        
    return np.array(cv_errors)
    
    
def prediction_int_coverage_cv(cv_test, cv_intervals):
    cv_test = np.array(cv_test)
    cv_intervals = np.array(cv_intervals)
    n_horizons = len(cv_test)    
    
    horizon_coverage = []
    for h in range(n_horizons):
        split_coverages = split_coverage(cv_test[h], cv_intervals[h])
        horizon_coverage.append(split_coverages)

    return np.array(horizon_coverage)  

In [28]:
def split_cv_error_scaled(cv_preds, cv_test, y_train):
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = mean_absolute_scaled_error(cv_test[split], cv_preds[split], 
                                                y_train, period=7)
        
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv_scaled(cv_preds, cv_test, y_train):
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error_scaled(cv_preds[h], cv_test[h], y_train)
        horizon_errors.append(split_errors)
        
    return np.array(horizon_errors)

In [29]:
def get_ensemble(fb_interval=0.8):
    '''
    Create and return ensemble model
    '''
    model_1 = ARIMAWrapper(order=(1,1,3), seasonal_order=(1,0,1,7), 
                           training_index=train.index,
                           holidays=new_year['ds'].tolist())

    model_2 = ExponentialSmoothingWrapper(trend=True, damped_trend=True, 
                                          seasonal=7)
    
    estimators = {'arima': model_1, 'hw': model_2}
    return Ensemble(estimators, UnweightedVote())
    

# Run cross validation

This is run twices once each for 80 and 95% prediction intervals.  

In [30]:
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
model = get_ensemble()

results = time_series_cv(model, train[REGION], val[REGION], horizons, 
                         alpha=0.2, step=7)

split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



# symmetric MAPE results

In [31]:
cv_preds, cv_test, cv_intervals = results
#CV point predictions smape
cv_errors = forecast_errors_cv(cv_preds, cv_test, 
                               symmetric_mean_absolute_percentage_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,2.841716,3.193180,3.346955,3.462023,3.610849,3.748776,3.847361,3.938023,4.043647,4.110324,4.170022,4.251676,5.108226
std,1.092489,1.188468,1.147059,1.060012,1.100819,1.127005,1.140626,1.108010,1.106966,1.010946,0.938292,0.974988,1.652565
min,1.498201,1.735182,1.875330,2.141260,2.214448,2.357114,2.374147,2.493206,2.690794,2.889782,2.943369,3.072235,3.488983
25%,1.960857,2.513679,2.694034,2.819100,2.837185,2.979749,3.078592,3.247460,3.409912,3.420824,3.468915,3.627075,3.777043
50%,2.444450,2.891852,3.125479,3.256980,3.367021,3.324309,3.626755,3.710027,3.851025,3.991769,4.051374,4.029532,5.049119
75%,3.639793,3.777411,3.725987,3.709215,4.211614,4.456798,4.493621,4.384990,4.424759,4.308067,4.683766,4.773524,5.863021
max,5.634527,6.860175,7.351093,7.107138,7.532948,7.603609,7.607221,7.323897,7.679503,7.395063,7.156615,7.537536,11.351217


In [32]:
#output sMAPE results to file
metric = 'smape'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-arima-hw_smape.csv


# RMSE results

In [33]:
#CV point predictions rmse
cv_errors = forecast_errors_cv(cv_preds, cv_test, root_mean_squared_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,74.988263,85.534422,91.410668,95.445099,100.067545,104.244589,107.463300,110.352160,113.669126,116.195505,118.397852,121.176639,142.623856
std,28.434099,33.140740,33.058368,31.452604,32.111216,32.239890,32.536045,32.215778,31.899325,29.545289,27.485643,27.830162,37.574007
min,46.797529,44.698958,49.339830,59.359345,61.101241,65.834922,66.083921,68.545492,74.561462,79.081271,80.194306,82.736560,106.438723
25%,53.318795,65.185142,73.550448,79.750552,78.776548,79.661766,82.186630,88.268085,93.351392,91.701330,94.584957,100.402162,113.759928
50%,64.816353,80.111237,80.686719,84.297659,89.619708,91.289293,96.552269,99.144146,104.186922,111.671527,117.501815,116.134107,140.977670
75%,87.712965,95.468675,94.728276,94.631052,108.123192,123.185961,130.726748,129.125872,127.556611,127.305474,130.351363,138.110855,159.353892
max,156.956142,183.112775,190.129699,182.786288,191.532560,191.928067,191.950803,186.175786,194.533736,189.502333,184.414480,195.756517,286.424396


In [34]:
#output RMSE to file
metric = 'rmse'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-arima-hw_rmse.csv


# Mean Absolute Scaled Error (MASE)

Scaled by one-step insample Seasonal Naive

In [35]:
#mase
cv_errors = forecast_errors_cv_scaled(cv_preds, cv_test, train[REGION])
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.767222,0.865183,0.909189,0.942009,0.983500,1.022064,1.049843,1.074886,1.104023,1.122465,1.139132,1.161788,1.387167
std,0.325144,0.359754,0.350113,0.328241,0.338548,0.344636,0.346127,0.335549,0.333356,0.304900,0.282118,0.288518,0.468986
min,0.408913,0.457874,0.503834,0.576618,0.599030,0.632419,0.638874,0.668845,0.725274,0.779323,0.760998,0.798394,0.949530
25%,0.542749,0.648681,0.711453,0.733531,0.740555,0.775785,0.807740,0.865963,0.909666,0.885829,0.920054,0.976661,1.025336
50%,0.652615,0.754847,0.812946,0.875076,0.874533,0.876260,0.959082,0.973048,1.041685,1.079482,1.117149,1.080794,1.371037
75%,0.944688,0.985114,0.992906,0.980608,1.129034,1.226786,1.246211,1.206528,1.228042,1.206092,1.293715,1.305888,1.570518
max,1.659366,2.001984,2.118957,2.045908,2.158184,2.175243,2.172599,2.090264,2.186301,2.105705,2.040416,2.145312,3.218237


In [36]:
#output mase to file.
metric = 'mase'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-arima-hw_mase.csv


# 80% Prediction Interval Coverage

In [37]:
#80% PIs
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.825397,0.820106,0.828924,0.838624,0.840212,0.841270,0.846561,0.854497,0.857143,0.860847,0.863877,0.865079,0.935667
std,0.207146,0.191428,0.165622,0.139862,0.140029,0.137094,0.129737,0.119246,0.122478,0.108406,0.097610,0.099766,0.057996
min,0.142857,0.142857,0.190476,0.285714,0.257143,0.261905,0.306122,0.392857,0.365079,0.428571,0.480519,0.464286,0.671233
25%,0.714286,0.750000,0.785714,0.821429,0.785714,0.797619,0.795918,0.803571,0.825397,0.828571,0.831169,0.833333,0.936986
50%,0.857143,0.857143,0.857143,0.892857,0.885714,0.880952,0.897959,0.892857,0.888889,0.885714,0.870130,0.869048,0.945205
75%,1.000000,0.928571,0.928571,0.910714,0.914286,0.928571,0.928571,0.928571,0.928571,0.921429,0.922078,0.928571,0.963014
max,1.000000,1.000000,1.000000,1.000000,0.971429,0.976190,0.979592,0.982143,0.984127,0.985714,0.987013,0.976190,0.969863


In [38]:
#write 80% coverage to file
metric = 'coverage_80'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-arima-hw_coverage_80.csv


# 95% Prediction Interval Coverage

Rerun analysis and obtain 95% Prediction intervals

In [39]:
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
model = get_ensemble(fb_interval=0.95)

results = time_series_cv(model, train[REGION], val[REGION], horizons, 
                         alpha=0.05, step=7)

split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



In [40]:
#95% PIs
cv_preds, cv_test, cv_intervals = results
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.968254,0.952381,0.950617,0.953704,0.953439,0.955908,0.956160,0.957011,0.957672,0.959259,0.960558,0.959877,0.982750
std,0.072339,0.106684,0.107686,0.090141,0.079696,0.070105,0.063964,0.057813,0.054276,0.049281,0.044478,0.042560,0.008283
min,0.714286,0.571429,0.571429,0.678571,0.714286,0.761905,0.775510,0.803571,0.809524,0.828571,0.844156,0.833333,0.950685
25%,1.000000,0.928571,0.952381,0.964286,0.957143,0.952381,0.908163,0.892857,0.904762,0.914286,0.922078,0.928571,0.980822
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.982143,0.984127,0.985714,0.987013,0.976190,0.983562
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.986301
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.994521


In [41]:
#write 95% coverage to file
metric = 'coverage_95'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-arima-hw_coverage_95.csv


# End